In [4]:
import os
import re
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
# Get the list of all files and directories
path_1 = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\NER\best\Run_1"
path_2 = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\NER\best\Run_2"
path_3 = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\NER\best\Run_3"
path_4 = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\NER\best\Run_4"
path_5 = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\NER\best\Run_5"

#ablation_path = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\NER\Ablation\Run_1"
dir_list = os.listdir(path)
#ablation_dir_list = os.listdir(ablation_path)

In [18]:
dir_list

['Aug_size_0_Epochs_30_Aug_Model_DCGAN_v2_Text_random_accuracy.txt',
 'Aug_size_100_Epochs_30_Aug_Model_DCGAN_v2_Text_random_accuracy.txt',
 'Aug_size_100_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-High_accuracy.txt',
 'Aug_size_100_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-Low_accuracy.txt',
 'Aug_size_100_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-Medium_accuracy.txt',
 'Aug_size_10_Epochs_30_Aug_Model_DCGAN_v2_Text_random_accuracy.txt',
 'Aug_size_10_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-High_accuracy.txt',
 'Aug_size_10_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-Low_accuracy.txt',
 'Aug_size_10_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-Medium_accuracy.txt',
 'Aug_size_15_Epochs_30_Aug_Model_DCGAN_v2_Text_random_accuracy.txt',
 'Aug_size_15_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-High_accuracy.txt',
 'Aug_size_15_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-Low_accuracy.txt',
 'Aug_size_15_Epochs_30_Aug_Model_DCGAN_v2_Text_TF-IDF-Medium_accuracy.txt',
 'Aug_size_20_Epochs_30_Aug_Model_DCGAN

In [19]:
def read_file(read_in_path):
    #Open the text file for reading
    with open(read_in_path, 'r') as file:
        # Read the content of the file
        content = file.read()

        # Search for the line containing "Test Accuracy:"
        line = content.split('\n')
        for l in line:
            if "Test Accuracy:" in l:
                # Extract the value following "Test Accuracy:"
                accuracy_str = l.split(':')[1].strip()
                # Convert the extracted value to float
                accuracy = float(accuracy_str)
                break

    return accuracy

In [20]:
def create_df(list_of_dir, read_in_path):
    list_of_dfs = []
    for dir in list_of_dir:
        # Get the full path of the file
        file_path = os.path.join(read_in_path, dir)

        if "15" in dir:
            aug_size = 15
        elif "25" in dir:
            aug_size = 25
        elif "50" in dir:
            aug_size = 50
        elif "75" in dir:
            aug_size = 75
        elif "100" in dir:
            aug_size = 100
        elif "10" in dir:
            aug_size = 10
        elif "5" in dir:
            aug_size = 5
        elif "20" in dir:
            aug_size = 20
        else:
            aug_size = 0

        if "Noise" in dir:
            model = "Noise"
        elif "DCGAN_v2_Text" in dir:
            model = "DCGAN_v2_Text"

        if "random" in dir:
            aug_type = "random"
        elif "TF-IDF-Low" in dir:
            aug_type = "TF-IDF-Low"
        elif "TF-IDF-High" in dir:
            aug_type = "TF-IDF-High"
        elif "TF-IDF-Medium" in dir:
            aug_type = "TF-IDF-Medium"
        else:
            aug_type = "None"

        # Read the file
        accuracy = read_file(file_path)

        df = pd.DataFrame([accuracy], columns=["accuracy"])
        df["augmentation_size"] = aug_size
        df["model"] = model
        df["augmentation_type"] = aug_type

        list_of_dfs.append(df)

    return pd.concat(list_of_dfs)

In [21]:
#create ablation and normal df
df_1 = create_df(dir_list, path_1)
df_2 = create_df(dir_list, path_2)
df_3 = create_df(dir_list, path_3)
df_4 = create_df(dir_list, path_4)
df_5 = create_df(dir_list, path_5)



In [30]:
concatenated_df = pd.concat([df_1, df_2, df_3, df_4, df_5], ignore_index=True)
# Group by augmentation size, augmentation type, and model, and calculate the average accuracy and standard deviation
grouped_df = concatenated_df.groupby(['augmentation_size', 'augmentation_type', 'model'])['accuracy'].agg(['mean', 'std']).reset_index()
# Rename columns for clarity
grouped_df.rename(columns={'mean': 'avg_accuracy', 'std': 'std_accuracy'}, inplace=True)
